In [1]:
%env CUDA_VISIBLE_DEVICES=6

import math

import torch
from linear import HiggsLinear

env: CUDA_VISIBLE_DEVICES=6


In [2]:
SIZE = 8192
HADAMARD_SIZE = 1024
GRID_DIM = 2

layer = HiggsLinear(
    SIZE, SIZE,
    GRID_DIM, 256,
    device="cuda",
    dtype=torch.float16,
)

In [3]:
input = torch.rand(1, 1, SIZE, device="cuda", dtype=torch.float16)

layer(input)

tensor([[[  8.6797,  19.6562, -51.1250,  ...,  13.0625, -24.7344, -42.0625]]],
       device='cuda:0', dtype=torch.float16)

In [4]:
from aqlm.utils import unpack_int_data, _dequantize_weight

In [5]:
@torch.inference_mode()
def unpack_int_data(data: torch.IntTensor, nbits: int) -> torch.IntTensor:
    return data.to(torch.int64) % (2**nbits)

codebooks = torch.load(f"../grids/EDEN{GRID_DIM}-256.pt").half()

weight = _dequantize_weight(
    unpack_int_data(layer.codes, 8)[:,:,None],
    codebooks[None,:,None,:],
    # scales=layer.scales,
).reshape(-1, SIZE//HADAMARD_SIZE, HADAMARD_SIZE) * layer.scales[...,None]

weight = weight.reshape(-1, SIZE)

In [6]:
from fast_hadamard_transform import hadamard_transform

hadamard_transform(input.reshape(-1, HADAMARD_SIZE), scale=1/math.sqrt(HADAMARD_SIZE)).reshape(1, -1) @ weight.T

tensor([[  8.7031,  19.6562, -51.0938,  ...,  13.0859, -24.7344, -42.0625]],
       device='cuda:0', dtype=torch.float16)

In [7]:
%%time

with torch.inference_mode():
    for i in range(1000):
        layer(input)
        torch.cuda.synchronize()

CPU times: user 206 ms, sys: 1.33 ms, total: 207 ms
Wall time: 206 ms


In [8]:
%%time

with torch.inference_mode():
    for i in range(1000):
        torch.nn.functional.linear(input, weight)
        torch.cuda.synchronize()

CPU times: user 182 ms, sys: 0 ns, total: 182 ms
Wall time: 182 ms
